In [1]:
import re, os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
from glob import glob
from dapeng.models.unet import unet5
from dapeng.utils.keras_tools import ImageDirectoryIterator, x_tif_reader, predict_from_directory
% matplotlib inline

Using TensorFlow backend.


In [2]:
input_path = "/root/input"
output_path = "/root/output"
dir_range = (108694, 108924)

In [3]:
model = unet5(256)
model.load_weights("ModelArchive/release_v1.0.hdf5")

In [4]:
def imsave(savepath, array):
    Image.fromarray(array, mode="L").save(savepath)

In [6]:
for row in tqdm_notebook(range(*dir_range), desc=u"预测"):
    glob_path = os.path.join(input_path, "{}/*.jpeg".format(row))
    filepaths = glob(glob_path)
    gen = ImageDirectoryIterator([(filepath, None) for filepath in filepaths], x_tif_reader, None, batch_size=16,
                                 shuffle=False)
    predic_y = model.predict_generator(gen, steps=len(gen), verbose=0)
    predic_y_byte = (predic_y * 255).astype(np.uint8)
    for i, filepath in enumerate(filepaths):
        filename = os.path.basename(filepath)
        gray = predic_y_byte[i].reshape(256, 256)
        imsave(os.path.join(output_path, str(row) + "_" + os.path.splitext(filename)[0] + ".png"), gray)

Widget Javascript not detected.  It may not be installed or enabled properly.


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/mac/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/mac/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/mac/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/keras/utils/data_utils.py", line 568, in data_generator_task
    generator_output = next(self._generator)
  File "/Users/mac/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/dapeng/utils/keras_abc.py", line 149, in next
    index_array = next(self.index_generator)
  File "/Users/mac/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/dapeng/utils/keras_abc.py", line 127, in _flow_index
    current_index = (self.batch_index * self.batch_size) % self.n
ZeroDivisionError: integer division or modulo by zero



AttributeError: 'list' object has no attribute 'astype'